# Intracellular Electrophysiology in NWB

### Creating and Writing NWB files

First step is to create a `NWBFile` object to store the data.

In [2]:
from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile
from hashlib import sha256
import numpy as np

# timestamp of the zero of the file level clock
now = datetime.now(tzlocal())

# random identifier, should be unique across all NWB files worldwide
my_id = sha256(f'Our Lab identifier: {now}'.encode()).hexdigest()


nwbfile = NWBFile(session_description='My first synthetic IC ephys recording', 
                  identifier=my_id, 
                  session_start_time=now,
                  experimenter='Dr. Bilbo Baggins',
                  lab='Bag End Laboratory',
                  institution='University of Middle Earth at the Shire',
                  experiment_description='I went on an adventure with thirteen dwarves to reclaim vast treasures.',
                  session_id='LONELYMTN')

### Device and Electrode metadata

<img src="images/IntracellularElectrode.svg" width="200">

Device metadata is represented by `Device` objects. To create a device, you can use the `Device` instance method `NWBFile.create_device`.

In [3]:
device = nwbfile.create_device(name='Heka ITC-1600')
device

Heka ITC-1600 pynwb.device.Device at 0x4396018032

Intracellular electrode metadata is represented by `IntracellularElectrode` objects.
To create an electrode, you can use the `NWBFile` instance method `NWBFile.create_icephys_electrode`.


In [4]:
elec0 = nwbfile.create_icephys_electrode(name="elec0",
                                         description='An intracellular electrode',
                                         device=device,
                                         slice='Bath solution MIBS2', 
                                         seal='2e09')
elec0

elec0 pynwb.icephys.IntracellularElectrode at 0x4805110608
Fields:
  description: An intracellular electrode
  device: Heka ITC-1600 pynwb.device.Device at 0x4396018032
  seal: 2e09
  slice: Bath solution MIBS2

### Adding Stimulus and Response Traces

Intracellular stimulus and response data are represented with subclasses of `PatchClampSeries`. There are two classes for representing stimulus data:

- `VoltageClampStimulusSeries`
- `CurrentClampStimulusSeries`

Here, we will use`CurrentClampStimulusSeries` to store current clamp stimulus data, and then add it to our `NWBFile` as a stimulus data trace using the method `NWBFile.add_stimulus`.

<img src="images/VCStimSeries_no_TS.svg" width="600">

In [5]:
from pynwb.icephys import CurrentClampStimulusSeries

ccss0 = CurrentClampStimulusSeries(name="CC_Stim_Sweep0", 
                                  data=[1, 2, 3, 4, 5], 
                                  starting_time=123.6, 
                                  rate=10e3, 
                                  electrode=elec0, 
                                  gain=0.02, 
                                  sweep_number=0)

nwbfile.add_stimulus(ccss0)

### Sweeps and Sweep Numbers

Sweep numbers are currently used in NWB to group stimulus and response traces, an are assigned by the user when the `PatchClampSeries` is created. `PatchClampSeries` having the same starting time belong to the same sweep. These can have the same or different `Electrode` objects, e.g. for multi-patching in the same slice. For now, we will add another stimulus series belonging a different sweep.


In [6]:
from pynwb.icephys import VoltageClampStimulusSeries

vcss1 = VoltageClampStimulusSeries(name="VC_Stim_Sweep1",
                                  data=[2, 3, 4, 5, 6],
                                  starting_time=234.5,
                                  rate=10e3, 
                                  electrode=elec0, 
                                  gain=0.03, 
                                  sweep_number=1)

nwbfile.add_stimulus(vcss1)

<img src="images/VCClampSeries_no_TS.svg" width="600">

We have three classes for representing responses:

- `VoltageClampSeries`
- `CurrentClampSeries`
- `IZeroClampSeries`


Here, we will use `CurrentClampSeries` to store current clamp data from sweep 0 and add it to our NWBFile as response data using the method `NWBFile.add_acquisition`.

In [7]:
from pynwb.icephys import CurrentClampSeries

ccs0 = CurrentClampSeries(name="CC_Resp_Sweep0",
                         data=[0.1, 0.2, 0.3, 0.4, 0.5],
                         conversion=1e-12, 
                         resolution=np.nan, 
                         starting_time=123.6, 
                         rate=20e3,
                         electrode=elec0, 
                         gain=0.02, 
                         bias_current=1e-12, 
                         bridge_balance=70e6,
                         capacitance_compensation=1e-12, 
                         sweep_number=0)

nwbfile.add_acquisition(ccs0)

Now add voltage clamp data from sweep 1 using `VoltageClampSeries` and `NWBFile.add_acquisition`.

In [8]:
from pynwb.icephys import VoltageClampSeries

vcs1 = VoltageClampSeries(name="VC_Resp_Sweep1", 
                         data=[0.1, 0.2, 0.3, 0.4, 0.5],
                         conversion=1e-12, 
                         resolution=np.nan, 
                         starting_time=234.5, 
                         rate=20e3,
                         electrode=elec0, 
                         gain=0.02, 
                         capacitance_slow=100e-12, 
                         resistance_comp_correction=70.0,
                         sweep_number=1)

nwbfile.add_acquisition(vcs1)

Once you have finished adding all of your data to the `NWBFile`, write the file with `NWBHDF5IO`.


In [9]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('My_First_Icephys_File.nwb', 'w') as io:
    io.write(nwbfile)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/hdmf/build/objectmapper.py:145: UserWarning: Value with data type int64 is being converted to data type uint64 (min specification: uint32).
  warnings.warn('Value with data type %s is being converted to data type %s (min specification: %s).'


## Reading data

Now that you have written some intracellular electrophysiology data to an NWB file, you can read it back in. For additional details on features for retrieving `TimeSeries` data from an `NWBFile`, we refer the reader to the basic tutorial. We will cover the basics for loading data here, including retrieving sweeps from the `SweepTable` using sweep numbers.

In [10]:
io = NWBHDF5IO('My_First_Icephys_File.nwb', 'r')
read_nwbfile = io.read()

### Retrieving individual stimulus and response traces

First, we can examine the file to see what stimulus traces are stored, and then retrieve the `CurrentClampStimulusSeries` we added as stimulus data using the `NWBFile.get_stimulus` method:

In [11]:
stims = read_nwbfile.stimulus
for i in stims:
    print(i)

CC_Stim_Sweep0
VC_Stim_Sweep1


In [12]:
read_ccss0 = read_nwbfile.get_stimulus('CC_Stim_Sweep0')
read_ccss0.data[()]

array([1, 2, 3, 4, 5])

Grabbing response data can be done via `NWBFile.get_acquisition`. First we can check the list of responses in the file  by reading out the keys from `NWBFile.acquisition`:

In [13]:
list(read_nwbfile.acquisition.keys())

['CC_Resp_Sweep0', 'VC_Resp_Sweep1']

In [14]:
vcs = read_nwbfile.get_acquisition('VC_Resp_Sweep1')
vcs.data[()]

array([0.1, 0.2, 0.3, 0.4, 0.5])

### Retrieving electrode and device metadata

We can access the `NWBFile.icephys_electrodes` attribute to return a list of the intracellular electrodes in this NWB file:

In [15]:
read_nwbfile.icephys_electrodes

{'elec0': elec0 pynwb.icephys.IntracellularElectrode at 0x4805110944
 Fields:
   description: An intracellular electrode
   device: Heka ITC-1600 pynwb.device.Device at 0x4805108304
   seal: 2e09
   slice: Bath solution MIBS2}

We can retrieve a specific electrode by `name` using the `NWBFile.get_icephys_electrode` method:

In [16]:
elec = nwbfile.get_icephys_electrode('elec0')

Alternatively, we can also get this electrode from the `CurrentClampStimulusSeries` we retrieved above. This is exposed via the `PatchClampSeries.electrode` attribute:


In [17]:
pcs_elec0 = read_ccss0.electrode
pcs_elec0

elec0 pynwb.icephys.IntracellularElectrode at 0x4805110944
Fields:
  description: An intracellular electrode
  device: Heka ITC-1600 pynwb.device.Device at 0x4805108304
  seal: 2e09
  slice: Bath solution MIBS2

We can see the names of all the `Devices` in this NWB file using the `NWBFile.devices` attribute: 

In [18]:
read_nwbfile.devices

{'Heka ITC-1600': Heka ITC-1600 pynwb.device.Device at 0x4805108304}

And we can retrieve specific device metadata by name using `NWBFile.get_device`:

In [19]:
device = read_nwbfile.get_device('Heka ITC-1600')
device.name

'Heka ITC-1600'

### Retrieving data grouped by Sweep

If you have data from multiple electrodes and multiple sweeps, it can be tedious and expensive to search all `PatchClampSeries` for the `TimeSeries` from a given sweep.

Fortunately you don't have to do that manually, instead you can just query the `NWBFile.SweepTable` which stores the mapping between the `PatchClampSeries` which belong to a certain sweep number via `SweepTable.get_series`.


In [20]:
read_nwbfile.sweep_table

sweep_table pynwb.icephys.SweepTable at 0x4802719216
Fields:
  colnames: ['series' 'sweep_number']
  columns: (
    series_index <class 'hdmf.common.table.VectorIndex'>,
    series <class 'hdmf.common.table.VectorData'>,
    sweep_number <class 'hdmf.common.table.VectorData'>
  )
  description: A sweep table groups different PatchClampSeries together.
  id: id <class 'hdmf.common.table.ElementIdentifiers'>

To retrieve a list of the sweep numbers in a `SweepTable`, we can use `SweepTable.sweep_number`. 
Note it will return a value for each individual `PatchClampSeries` stored.

In [21]:
sweep_nums = read_nwbfile.sweep_table.sweep_number
print(np.unique(sweep_nums[()]))

[0 1]


Now we can retrieve the `PatchClampSeries` for sweep 0 using the `NWBFile.sweep_table.get_series` method:

In [22]:
pcs0 = nwbfile.sweep_table.get_series(0)
pcs0

[CC_Stim_Sweep0 pynwb.icephys.CurrentClampStimulusSeries at 0x4805111664
 Fields:
   comments: no comments
   conversion: 1.0
   data: [1 2 3 4 5]
   description: no description
   electrode: elec0 pynwb.icephys.IntracellularElectrode at 0x4805110608
 Fields:
   description: An intracellular electrode
   device: Heka ITC-1600 pynwb.device.Device at 0x4396018032
   seal: 2e09
   slice: Bath solution MIBS2
 
   gain: 0.02
   rate: 10000.0
   resolution: -1.0
   starting_time: 123.6
   starting_time_unit: seconds
   stimulus_description: NA
   sweep_number: 0
   unit: amperes,
 CC_Resp_Sweep0 pynwb.icephys.CurrentClampSeries at 0x4396017024
 Fields:
   bias_current: 1e-12
   bridge_balance: 70000000.0
   capacitance_compensation: 1e-12
   comments: no comments
   conversion: 1e-12
   data: [0.1 0.2 0.3 0.4 0.5]
   description: no description
   electrode: elec0 pynwb.icephys.IntracellularElectrode at 0x4805110608
 Fields:
   description: An intracellular electrode
   device: Heka ITC-1600

We can also us the `SweepTable.to_dataframe` method to convert the sweep table to a dataframe that we can inspect and access to retrieve sweep numbers, stimulus series and response series.

In [23]:
# Viewing sweep table as a pandas dataframe
stdf = read_nwbfile.sweep_table.to_dataframe()
stdf

,series,sweep_number
id,,
0,[CC_Stim_Sweep0 pynwb.icephys.CurrentClampStim...,0
1,[VC_Stim_Sweep1 pynwb.icephys.VoltageClampStim...,1
2,[CC_Resp_Sweep0 pynwb.icephys.CurrentClampSeri...,0
3,[VC_Resp_Sweep1 pynwb.icephys.VoltageClampSeri...,1


In [24]:
# To retrieve a list of sweep numbers
print(list(stdf.sweep_number))

[0, 1, 0, 1]


In [25]:
stdf.series[1][0].data[()]

array([2, 3, 4, 5, 6])

In [26]:
stdf.series[0][0].electrode

elec0 pynwb.icephys.IntracellularElectrode at 0x4805110944
Fields:
  description: An intracellular electrode
  device: Heka ITC-1600 pynwb.device.Device at 0x4805108304
  seal: 2e09
  slice: Bath solution MIBS2